In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

# 7.1 Handling Missing Data

자동적으로 pandas에서 missing value(NaN)은 제외하고 처리한다. NaN은 sentinel value라 함(easily detected)

.isnull로 쉽게 파악할 수 있음

python자체의 None도 NaN 역할

In [3]:
# .dropna: na 뺀 data만 남기고 호출
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [4]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [6]:
# dropna는 na 있는 행은 다 없앤다
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [7]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


In [8]:
# how='all' 쓰면 모두 na인 행만 없앰
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [9]:
# na를 열 기준으로 없애려면 axis=1추가
data[4] = NA
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [11]:
# thresh를 쓰면 일정 value를 갖고 있는 row만 남길 수 있음
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.260028,NaN,NaN
1,-0.621855,NaN,NaN
2,-1.501764,NaN,1.387116
3,0.501616,NaN,1.040905
4,-1.220720,0.648152,0.626583
5,0.051609,0.346123,-1.061701
6,0.442110,-0.780833,-0.159329


In [12]:
df.dropna(thresh=2)

,0,1,2
2,-1.501764,NaN,1.387116
3,0.501616,NaN,1.040905
4,-1.220720,0.648152,0.626583
5,0.051609,0.346123,-1.061701
6,0.442110,-0.780833,-0.159329


In [13]:
df.dropna(thresh=1)

,0,1,2
0,-0.260028,NaN,NaN
1,-0.621855,NaN,NaN
2,-1.501764,NaN,1.387116
3,0.501616,NaN,1.040905
4,-1.220720,0.648152,0.626583
5,0.051609,0.346123,-1.061701
6,0.442110,-0.780833,-0.159329


In [ ]:
# fillna 로는 na인 값을 원하는 값으로 채울 수 있음
df.fillna({1: 0.5, 2: 0}) # 이렇게 열 별로 다른 값으로도 가능
# inplace로 기존 자료를 바꿔버린다
_ = df.fillna(0, inplace=True)
df

In [14]:
# index에서 했던거처럼 ffill 사용
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,0.669083,-0.578555,1.469586
1,0.336136,-0.934939,-0.995342
2,-0.789492,NaN,0.220570
3,-0.297191,NaN,-1.157936
4,0.698738,NaN,NaN
5,1.122076,NaN,NaN


In [15]:
df.fillna(method='ffill', limit=2) # 열 별 2개만 채워준다

,0,1,2
0,0.669083,-0.578555,1.469586
1,0.336136,-0.934939,-0.995342
2,-0.789492,-0.934939,0.220570
3,-0.297191,-0.934939,-1.157936
4,0.698738,NaN,-1.157936
5,1.122076,NaN,-1.157936


In [ ]:
# 이렇게 평균으로 채워주는건 많이 쓸 것 같다.
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

# 7.2 Data Transformation

In [16]:
# duplicated로 중복된 자료 찾기
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [17]:
# drop_duplicates 는 중복이 안된 자료만 출력
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [18]:
# 특정 열만 중복 체크 가능
data['v1'] = range(7)
data.drop_duplicates(['k1'])
# keep='last'를 쓰면 첫번쨰 자료가 아니라 마지막 자료만 남음

,k1,k2,v1
0,one,1,0
1,two,1,1


In [22]:
# data에 mapping 적용하기
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
# 이 데이터를 다음 dict에 맞게 바꾸고 싶다
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [23]:
# 우선 대소문자가 섞여 있으니 바꿔주자.
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [24]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [25]:
# 아니면 만능 lambda 함수를 이용해도 됨
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [26]:
# 이번엔 replace로 missing value 채우기
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data.replace([-999,-1000], np.nan) # -999, -1000를 nan으로 바꾼다
# 아예 replace({dictionary})꼴로 넣어도됨

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [28]:
# 이번엔 index 바꾸기
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [30]:
transform = lambda x: x[:4].upper()
data.index=data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [31]:
# 바꾸기 전 거 말고 새로 바뀐 data 만들고 싶으면 rename
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [32]:
# 이렇게 dict 형식으로 대응시킬 수도 있음
data.rename(index={'OHIO': 'INDIANA'},columns={'three': 'peekaboo'})
# 새로 바뀐걸로 기존 data 바꿔버리려면 inplace=True

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [33]:
# data를 범주화하려면 cut
# bin 기준으로 age를 나눠준다
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [35]:
#cut 된 결과의 attribute
cats.codes #몇번쨰 범주에 속한지

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [36]:
cats.categories # 전체 범주의 종류

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [37]:
# value_counts로 범주당 몇개씩 있는지 파악
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [41]:
# cut 할 때 right=False 하면 초과 이하 가 바뀜. (이상 미만으로)
# cut 할 때 labels=[]로 bin의 이름 설정 가능
# bin 대신 정수n 넣으면 bin을 같은 간격으로 나눔
data = np.random.rand(1000)  # 정규분포
cats = pd.cut(data, 4, precision=2)  # 4등분, 소수점 2자리까지
pd.value_counts(cats)

(0.0014, 0.25]    262
(0.25, 0.5]       248
(0.75, 1.0]       247
(0.5, 0.75]       243
dtype: int64

In [43]:
# bin당 data를 같게 하려면 qcut으로
cat2=pd.qcut(data, 4)
pd.value_counts(cat2)

(0.742, 0.998]                    250
(0.491, 0.742]                    250
(0.241, 0.491]                    250
(0.0014099999999999998, 0.241]    250
dtype: int64

In [44]:
# quantile도 설정가능
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(0.107, 0.491], (0.491, 0.888], (0.491, 0.888], (0.107, 0.491], (0.0014099999999999998, 0.107], ..., (0.107, 0.491], (0.107, 0.491], (0.107, 0.491], (0.888, 0.998], (0.107, 0.491]]
Length: 1000
Categories (4, interval[float64]): [(0.0014099999999999998, 0.107] < (0.107, 0.491] < (0.491, 0.888] < (0.888, 0.998]]

In [45]:
# 이번엔 oulier 잡아서 처리하기
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe() # 대략 요약해준다

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.006856,0.000679,-0.014796,0.022311
std,0.990697,0.977192,0.981536,1.042174
min,-4.060795,-2.671671,-3.664741,-3.328637
25%,-0.649551,-0.660543,-0.663459,-0.702116
50%,-0.007253,-0.005668,-0.019407,-0.007733
75%,0.708994,0.658713,0.619078,0.762357
max,3.212197,3.419277,2.920404,3.044717


In [46]:
# 절댓값이 3보다 큰 걸 보자
data[(np.abs(data) > 3).any(1)] # any를 통해 4열 중 하나라도 그러면 출력

,0,1,2,3
140,0.668747,0.907300,0.160730,3.044717
383,0.490422,1.702336,-0.706909,3.013171
388,2.842009,0.010543,-3.011606,-0.822121
407,0.680133,0.789350,0.280664,-3.206858
474,-0.066005,3.419277,-0.317437,-1.118688
652,0.205532,0.223371,-3.385877,0.255745
729,-4.060795,-0.561265,0.152556,0.138263
753,-3.128711,-1.231606,-0.114901,-0.227403
821,3.212197,-2.051346,-0.520996,-0.665807
849,0.049936,-0.598656,0.046477,-3.328637


In [47]:
# sign을 쓰면 1 or -1로 나오니까
# 너무 큰 값을 3 또는 -3으로 바꾸자
data[np.abs(data) > 3] = np.sign(data) * 3

,0,1,2,3
0,-1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0
2,-1.0,-1.0,1.0,-1.0
3,-1.0,-1.0,1.0,1.0
4,1.0,-1.0,1.0,1.0
...,...,...,...,...
995,-1.0,1.0,-1.0,-1.0
996,-1.0,1.0,-1.0,-1.0
997,-1.0,-1.0,1.0,1.0
998,-1.0,1.0,1.0,-1.0


In [48]:
# permutation
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([2, 4, 1, 0, 3])

In [50]:
# take를 써서 sampler에서 바꾼 순서대로 출력
df.take(sampler)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15


In [51]:
# sample도 사용가능(비복원)
df.sample(n=3)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15


In [52]:
# 복원은 replace=True
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

0    5
1    7
0    5
3    6
1    7
1    7
4    4
3    6
1    7
3    6
dtype: int64

In [53]:
# dummy variable
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [54]:
# get_dummies로 key 별로 dummy 화 한다
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [55]:
# prefix화
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [56]:
# join을 써서 병합
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [58]:
# 이번엔 엄청 큰 data를 다뤄보자
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('/Statistics/pydata-book-2nd-edition/pydata-book-2nd-edition/datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

C:\Users\minzz\anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [59]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [60]:
# 우선 장르를 다 split해서 저장
all_genres

['Animation',
 "Children's",
 'Comedy',
 'Adventure',
 "Children's",
 'Fantasy',
 'Comedy',
 'Romance',
 'Comedy',
 'Drama',
 'Comedy',
 'Action',
 'Crime',
 'Thriller',
 'Comedy',
 'Romance',
 'Adventure',
 "Children's",
 'Action',
 'Action',
 'Adventure',
 'Thriller',
 'Comedy',
 'Drama',
 'Romance',
 'Comedy',
 'Horror',
 'Animation',
 "Children's",
 'Drama',
 'Action',
 'Adventure',
 'Romance',
 'Drama',
 'Thriller',
 'Drama',
 'Romance',
 'Thriller',
 'Comedy',
 'Action',
 'Action',
 'Comedy',
 'Drama',
 'Crime',
 'Drama',
 'Thriller',
 'Thriller',
 'Drama',
 'Sci-Fi',
 'Drama',
 'Romance',
 'Drama',
 'Drama',
 'Romance',
 'Adventure',
 'Sci-Fi',
 'Drama',
 'Drama',
 'Drama',
 'Sci-Fi',
 'Adventure',
 'Romance',
 "Children's",
 'Comedy',
 'Drama',
 'Drama',
 'Romance',
 'Drama',
 'Documentary',
 'Comedy',
 'Comedy',
 'Romance',
 'Drama',
 'Drama',
 'War',
 'Action',
 'Crime',
 'Drama',
 'Drama',
 'Action',
 'Adventure',
 'Comedy',
 'Drama',
 'Drama',
 'Romance',
 'Crime',
 'Thrill

In [62]:
# 이후 중복 제거
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [63]:
# 이젠 indicator matrix 만들기
zero_matrix = np.zeros((len(movies), len(genres)))
# 먼저 0행렬 만들기

In [69]:
# 이제 col은 장르고, 행렬은 0인 dataframe 만들기
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [73]:
# 이렇게 split하고
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [71]:
# split을 바탕으로 index 매기기
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

In [74]:
#행 별로 for 문을 돌려서 genre에 해당하는 칸에 1 넣기
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [75]:
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
# 합치기
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [77]:
# 이건 다른 예시
# 랜덤하게 숫자 생성
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [78]:
# 이렇게 균등한 bin 설정 후
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
# 이걸로 cut 한다음에 그걸 dummy화 한다
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


# 7.3 String Manipulation

In [79]:
# 여러가지 built in string method
# split: string을 지정한 delimiter로 나눠 list로 저장
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [80]:
# 이 이상한 blank를 없애는 게 strip
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [81]:
# join으로는 새로운 delimiter 만들어서 표현
'::'.join(pieces)

'a::b::guido'

In [82]:
# 포함되어 있는지 확인 할 땐 in, index, find
'guido' in val

True

In [83]:
val.index(',')

1

In [85]:
val.index(':')

ValueError: substring not found

In [86]:
val.find(',')

1

In [84]:
val.find(':')
# index와는 다르게 error가 안뜨고 -1로 뜬다

-1

In [87]:
# count로 개수도 세고
val.count(',')

2

In [88]:
# 바로 replace로 대체한다
val.replace(',', '::')
val.replace(',', '') # 없애기

'ab  guido'

In [90]:
# regular expression
# 크게 pattern matching, substitution, splitting
text = "foo    bar\t baz  \tqux"
re.split('\s+', text) # 한칸이상의 빈칸을 뜻하는 \s+

['foo', 'bar', 'baz', 'qux']

In [93]:
# 아예 이렇게 regex로 정해놓고 써도됨
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [94]:
# 빈칸을 찾으려면 findall
regex.findall(text)

['    ', '\t ', '  \t']

In [105]:
# search와 match
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [106]:
# 미리 지정한 pattern이 있는 string 출력
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [97]:
# 첫번째거만 출력하는 search
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [98]:
text[m.start():m.end()]

'dave@google.com'

In [99]:
# match는 string의 시작에 pattern있는지 확인
print(regex.match(text))

None


In [100]:
# sub는 pattern있는걸 바꿔준다
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [108]:
#메일 주소를 분리해보자
# 분리할 땐 이렇게 ()쓰기
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)
m = regex.match('wesm@bright.net')
m

<re.Match object; span=(0, 15), match='wesm@bright.net'>

In [109]:
m.groups()

('wesm', 'bright', 'net')

In [110]:
# 이걸 전체에 다 적용
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [111]:
# 분리한 대로 대응시킬수있음
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



In [112]:
# string이 na일때
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [113]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [114]:
# 앞에서 한게 na때문에 잘 안될거다.
# 이럴 땐 na를 skip하는데
# 바로 .str 을 써서!
data.str.contains('gmail') # na는 그냥 냅둔다!

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [115]:
# regular expression도 똑같다!
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [116]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [119]:
# str하고 이것저것 해보자
data.str.get(1)

Dave       a
Steve      t
Rob        o
Wes      NaN
dtype: object

In [120]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object